In [ ]:
import os
os.chdir('../')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from cnnclassifier.utils.common import read_dataset
import numpy as np
import tensorflow as tf
from cnnclassifier.components.model_building import model_building
from cnnclassifier.config.configuration import configManager

In [ ]:
%pwd

In [ ]:
print("Num GPUs Available: ", tf.config.list_physical_devices())


In [ ]:
print("CUDA_VISIBLE_DEVICES:", os.environ.get('CUDA_VISIBLE_DEVICES'))

In [ ]:
# build model
config = configManager()
m = model_building(config.get_build_model_config_params())
model = m.build_model()


In [ ]:
model.summary()

In [ ]:
# compile model
model = m.compile_model(model)

In [ ]:
# train nd validte the model
train_data = read_dataset("artifacts/split_data/train")
val_data = read_dataset("artifacts/split_data/validate")

from cnnclassifier.components.preprocessing import preprocessing
p = preprocessing(config.get_preprocess_data_config_params())
train_data = p.preprocess(train_data)
val_data = p.preprocess(val_data)

model, history = m.train_validate_model(model, train_data, val_data)

In [ ]:
import os
%pwd

In [ ]:
from cnnclassifier.utils.common import read_dataset
dataset = read_dataset('artifacts/split_data/train')

In [ ]:
# we will plot some of the images using subplot
for i, (image, label) in enumerate(dataset.take(25)):
    axes = plt.subplot(5,5, i+1)
    plt.imshow(image)
    plt.title(['infected' if label == 0 else 'safe'])
    plt.axis('off')


In [ ]:
# we will pratice resizing and normalizing
IMG_SIZE = 244
def resize_normalize(image,label):
    return tf.image.resize(image,[IMG_SIZE,IMG_SIZE])/255.0, label

__dataset = dataset.map(resize_normalize)
__dataset = __dataset.shuffle(32).batch(32).prefetch(tf.data.AUTOTUNE)

# the batch will divide the total data into the batch size

print(__dataset)


In [ ]:
count = 0
for image, label in dataset:
    if int(image.shape[0]) > 244 or int(image.shape[1]) > 244:
        # print( int(label), image.shape)
        # print('\n')
        count += 1
print(count)

In [ ]:
data = []
for image, label in dataset:
    height= image.shape[0]
    width = image.shape[1]
    channel = image.shape[2]
    label = int(label)
    max_val = int(np.max(image))
    min_val = int(np.min(image))

    data.append({
            'height':height,
            'width':width,
            'channel':channel,
            'label':label,
            'max_val':max_val,
            'min_val':min_val
        })


In [ ]:
daf = pd.DataFrame(data, columns=['height', 'width', 'channel','label','max_val','min_val'])

In [ ]:
daf.describe()

EDA on the images
1. The dtype of the images is 'uint8', which is is an 8-bit unsigned integer data type that can store values ranging from 0 to 255
2. from our train data set, i found out that there are 13 images with either the height > than 244 or width > 244 but all the channels for the images are 3. therefore wewill resize all the images to (244,244,3)
this can be done int the model or out side the model
3. Because the the image's numpy array ranges between 0 to 255, therefore we will normalize it to a range of 0 to 1
4. if the label for the imagfe is 0 that means infected but if is 1 that means safe